<a href="https://colab.research.google.com/github/mrzeszewski/Podstawy_Big_Data/blob/main/notatniki/Wprowadzenie_do_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Inicjalizacja GEE

In [2]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-mrzeszewski')

## Kolekcje danych

W GEE dane zabrane są w kolekcje obrazów (ImageCollection) i obiektów (FeatureCollection) - raster/wektor

In [17]:
# ImageCollection - warstwy rastrowe
populacja = ee.ImageCollection("WorldPop/GP/100m/pop") #kolekcja obrazów

mapka=geemap.Map() #incjalizacja mapy - wywołanie resetuje obiekt

#parametry wizualne w postaci słownika
vis_params={
    'min':0, #wartość rastra przypisana pierwszemu kolorowi z palety
    'max':5, #wartość rastra przypisana ostatniemu kolorowi z palety
    #'palette':['red','green','blue'] #piewszy sposób określenia palety - nazwy kolorów
    # 'palette':'inferno' #nazwa palety
    'palette':'viridis' #inna paleta
    #można też kolory szesnatkowo (RRGGBB lub RRGGBBAA) wg. CSS3
}

mapka.addLayer(populacja,vis_params=vis_params,name="Populacja") #dodanie warstwy do mapy

mapka #wyświetlenie mapy



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [20]:
#FeatureCollection - warstwy wektorowe

granice_panstw=ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")

mapka=geemap.Map() #incjalizacja mapy - wywołanie resetuje obiekt

vis_params={
    'color':'blue',
    'width':2,
    'lineType':'solid',
    'fillColor':'00000000'
}

mapka.addLayer(granice_panstw,vis_params=vis_params,name="Granice państw") #dodanie warstwy do mapy

mapka #wyświetlenie mapy

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…